### Premissas:
- Nome: dim_seller
- Descrição da tabela: Tabela composta por variáveis qualitativas de vendedores.
- Tipo: SCD-2

In [0]:
# Importa bibliotecas
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType, ArrayType
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from datetime import datetime

In [0]:
%sql
SELECT DISTINCT
  seller_id,
  count(*)
FROM poc_datum.olist.sellers_silver 
GROUP BY seller_id
HAVING count(*) > 1

seller_id,count(1)


In [0]:
# Cria dataframe com dados para a criação da dimensão
df_dimensao = spark.sql("""
SELECT DISTINCT
  seller_id,
  seller_zip_code_prefix,
  seller_city,
  seller_state
FROM poc_datum.olist.sellers_silver 
""")

display(df_dimensao.limit(10))

seller_id,seller_zip_code_prefix,seller_city,seller_state
f6122bc84774df1b372bdb3bb88ddb9f,6436,barueri,SP
8132b9bd16876e1b0f8808d43825dd48,88058,florianopolis,SC
a254c682cc01e119f83530446f1df9a9,16500,cafelandia,SP
cb9fb4ca75d7ba8437480e8dde64fe98,3551,sao paulo,SP
c5ebe6598748b0aeaa61cfb820478b92,95910,lajeado,RS
bd15ee794d5e640d9dd71b665b2ab15b,14078,ribeirao preto,SP
7994b065a7ffb14e71c6312cf87b9de2,29142,cariacica / es,ES
67883baaae6134ee81b271a542613728,1310,sao paulo,SP
f52c2422904463fdd7741f99045fecb6,9230,santo andre/sao paulo,SP
528bcf6680c36dddf07620bd35b33a6f,3178,sao paulo,SP


In [0]:
# Primeira carga

# Cria DataFrame com os dados de "NÃO INFORMADO"
data = [(-1, -1, "Não informado", "Não informado")]
column = df_dimensao.schema.fieldNames()
df = spark.createDataFrame( data, schema=column )
# df.show()

# Faz o join com o dataframe com dados para a criação da dimensão
dim_seller = df.union(df_dimensao)
# dim_customers.show()

# Inseri as colunas default de dimensão
dim_seller = dim_seller.withColumn( "row_ingestion_timestamp", current_timestamp() ) \
    .withColumn( "row_version", lit(1) ) \
    .withColumn( "row_current_indicator", lit(True) ) \
    .withColumn( "row_effective_date", to_timestamp( lit('1900-01-01 00:00:00'), "yyyy-MM-dd HH:mm:ss") ) \
    .withColumn( "row_expiration_date", to_timestamp( lit('2200-01-01 00:00:00') , "yyyy-MM-dd HH:mm:ss" ) )

# Inseri coluna SK
# dim_seller = dim_seller.withColumn( 'sk_dim_seller', monotonically_increasing_id() ) 
dim_seller = dim_seller.withColumn( 
    'sk_dim_seller', 
    sha2(concat_ws("|", 
        dim_seller.row_ingestion_timestamp,
        dim_seller.seller_id, 
        dim_seller.seller_zip_code_prefix), 256))

# Ordena as colunas
dim_seller_select = dim_seller.select( 
    'sk_dim_seller',
    'row_ingestion_timestamp',
    'row_version',
    'row_current_indicator',
    'row_effective_date',
    'row_expiration_date',
    'seller_id',
    'seller_zip_code_prefix',
    'seller_city',
    'seller_state'
    )

display(dim_seller_select.limit(10))

sk_dim_seller,row_ingestion_timestamp,row_version,row_current_indicator,row_effective_date,row_expiration_date,seller_id,seller_zip_code_prefix,seller_city,seller_state
e42d3c67e81ba8496f719126d75f9965b8168f4c90c8b35ff8c22b84298fc8d9,2024-04-29T15:45:33.431Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,-1,-1,Não informado,Não informado
f33b6e2d77f959081abfa7f3f5cabfdec9c7128d75f09db906ab00054213773f,2024-04-29T15:45:33.431Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,f6122bc84774df1b372bdb3bb88ddb9f,6436,barueri,SP
33380a454ab9fcbdbc0956cc1f7bc85afb9506d1b6d3ab05193eae59bd42d457,2024-04-29T15:45:33.431Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,8132b9bd16876e1b0f8808d43825dd48,88058,florianopolis,SC
8d047e777be3b3cb992abc2501e643a5b86dda149e6e29d2b2e22ecef48520af,2024-04-29T15:45:33.431Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,a254c682cc01e119f83530446f1df9a9,16500,cafelandia,SP
4dd12b602639465941410f437b2dbab6fa1034eb606b87310ef269b8d56239b0,2024-04-29T15:45:33.431Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,cb9fb4ca75d7ba8437480e8dde64fe98,3551,sao paulo,SP
abef0da68224f6650688a370aa86e6305a663707d37aa871db9a655e57e74d34,2024-04-29T15:45:33.431Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,c5ebe6598748b0aeaa61cfb820478b92,95910,lajeado,RS
b8e484f5b21b85d9b0dba9d0e4d95ce48fb430e3a908e80869967bcb7d6e288e,2024-04-29T15:45:33.431Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,bd15ee794d5e640d9dd71b665b2ab15b,14078,ribeirao preto,SP
7c4eddde996b18d6997ca7bcd56e46702919adb5a1140ba2081d99dd8a5a70ad,2024-04-29T15:45:33.431Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,7994b065a7ffb14e71c6312cf87b9de2,29142,cariacica / es,ES
bb1d8fd95691f591df0bd85b7813c9d7e765fe675269c8a76e2239c4e1b8f97a,2024-04-29T15:45:33.431Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,67883baaae6134ee81b271a542613728,1310,sao paulo,SP
104c2fdcab7b599f72ce313c3b1a761bf0800494121cf9b9e09dab8b88ebe632,2024-04-29T15:45:33.431Z,1,true,1900-01-01T00:00:00Z,2200-01-01T00:00:00Z,f52c2422904463fdd7741f99045fecb6,9230,santo andre/sao paulo,SP


In [0]:
table_name = 'dim_seller'
spark.sql('USE olist')
spark.sql(f'DROP TABLE IF EXISTS dim_seller')
dim_seller_select.write.format("delta").mode('overwrite').saveAsTable(table_name)

### Carga Diferencial (Upsert)

In [0]:
#teste
data = [('f6122bc84774df1b372bdb3bb88ddb9f',6436,'barueriL','SP')]
column = ['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state']

df_dimensao = spark.createDataFrame(data, schema=column)
df_dimensao.show()

+--------------------+----------------------+-----------+------------+
|           seller_id|seller_zip_code_prefix|seller_city|seller_state|
+--------------------+----------------------+-----------+------------+
|f6122bc84774df1b3...|                  6436|   barueriL|          SP|
+--------------------+----------------------+-----------+------------+



In [0]:
# Dados Novos
df_origem = df_dimensao

# Dados da dimensão
df_destino = spark.sql("""
    SELECT   
        seller_id,
        seller_zip_code_prefix,
        seller_city,
        seller_state 
    FROM 
        poc_datum.olist.dim_seller
    ORDER BY seller_id """)

# Realize o EXCEPT (retornar apenas registros novos)
df_dados_novos = df_origem.exceptAll(df_destino)

display(df_dados_novos)

# Cria uma tabela temporária
df_dados_novos.createOrReplaceTempView("temp_dados_novos")

seller_id,seller_zip_code_prefix,seller_city,seller_state
f6122bc84774df1b372bdb3bb88ddb9f,6436,barueriL,SP


In [0]:
# Paramêtros
table_merge = 'dim_seller'
change_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
print(change_date)

2024-04-29 15:45:42.945037


In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMPORARY VIEW dados_novos AS 
SELECT
     sha2(concat_ws("|",  '{change_date}',  a.seller_id, a.seller_zip_code_prefix), 256) AS sk_dim_seller
    ,a.seller_id
    ,a.seller_zip_code_prefix
    ,a.seller_city
    ,a.seller_state
    ,to_timestamp('{change_date}')  AS change_date
    ,(
        SELECT
            MAX(b.sk_dim_seller)
        FROM
            dim_seller as b
        WHERE
            a.seller_id = b.seller_id
    ) AS max_sk_dim_seller
    ,COALESCE(
        (
            SELECT
                MAX(c.row_version) + 1
            FROM
                dim_seller as c
            WHERE
                a.seller_id = c.seller_id
        ), 1
    ) AS max_row_version
FROM
    temp_dados_novos AS a
""")

DataFrame[]

In [0]:
%sql
SELECT
    *
FROM
    dados_novos AS a

sk_dim_seller,seller_id,seller_zip_code_prefix,seller_city,seller_state,change_date,max_sk_dim_seller,max_row_version
53fe025370fa736dba284e0086c255600603934018449a3cf3f4bbad684e4430,f6122bc84774df1b372bdb3bb88ddb9f,6436,barueriL,SP,2024-04-29T15:45:42.945037Z,7c8f628b130c3a381bb757f4b24c6e17b6f16d7b9cb65c499a071bb9e0176b95,2


In [0]:
spark.sql(f""" 
MERGE INTO {table_merge} as destino
USING dados_novos 
ON destino.sk_dim_seller = dados_novos.max_sk_dim_seller

WHEN MATCHED THEN 
  UPDATE SET
   destino.row_expiration_date = to_timestamp('{change_date}') --dados_novos.change_date
  ,destino.row_current_indicator = False
  """)

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
spark.sql(f"""
MERGE INTO {table_merge} as destino
USING dados_novos 
ON destino.seller_id = dados_novos.seller_id
AND destino.seller_city = dados_novos.seller_city

WHEN NOT MATCHED 
  THEN INSERT (
    sk_dim_seller
    , row_ingestion_timestamp
    ,row_version
    ,row_current_indicator
    ,row_effective_date
    ,row_expiration_date
    ,seller_id
    ,seller_zip_code_prefix
    ,seller_city
    ,seller_state
  )
  VALUES (
    dados_novos.sk_dim_seller
    ,to_timestamp('{change_date}') --dados_novos.change_date
    ,dados_novos.max_row_version
    ,1
    ,to_timestamp('{change_date}') --dados_novos.change_date
    ,to_timestamp( '2200-01-01 00:00:00')
    ,dados_novos.seller_id
    ,dados_novos.seller_zip_code_prefix
    ,dados_novos.seller_city
    ,dados_novos.seller_state
  )
""")

DataFrame[num_affected_rows: bigint, num_updated_rows: bigint, num_deleted_rows: bigint, num_inserted_rows: bigint]

In [0]:
%sql
SELECT * FROM poc_datum.olist.dim_seller
WHERE seller_id = 'f6122bc84774df1b372bdb3bb88ddb9f'

sk_dim_seller,row_ingestion_timestamp,row_version,row_current_indicator,row_effective_date,row_expiration_date,seller_id,seller_zip_code_prefix,seller_city,seller_state
53fe025370fa736dba284e0086c255600603934018449a3cf3f4bbad684e4430,2024-04-29T15:45:42.945037Z,2,true,2024-04-29T15:45:42.945037Z,2200-01-01T00:00:00Z,f6122bc84774df1b372bdb3bb88ddb9f,6436,barueriL,SP
7c8f628b130c3a381bb757f4b24c6e17b6f16d7b9cb65c499a071bb9e0176b95,2024-04-29T15:45:36.42Z,1,false,1900-01-01T00:00:00Z,2024-04-29T15:45:42.945037Z,f6122bc84774df1b372bdb3bb88ddb9f,6436,barueri,SP
